In [1]:
import pandas as pd
import numpy as np
from collections import Counter
import tensorflow as tf
%matplotlib inline

In [8]:
pre = pd.read_csv('pre_kd_cancer.csv')

pre_df = pd.DataFrame(pre)
pre_df

,person_id,gender,age,EGD,bmi,alcohol_habit,smoking status,cancer,date,observation_date
0,19905,1,29,NaN,NaN,1,NaN,1,2014-07-05,2007-06-29
1,36415,1,75,1.0,24.44,1,NaN,1,2017-02-23,2012-06-11
2,36415,1,75,1.0,24.44,1,NaN,1,2017-02-23,2012-06-11
3,36415,1,75,1.0,24.44,1,NaN,1,2017-02-23,2012-06-11
4,36415,1,75,1.0,24.44,1,NaN,1,2017-02-23,2012-06-11
...,...,...,...,...,...,...,...,...,...,...
6018,1737002,0,63,2.0,23.58,1,NaN,1,2009-03-04,2011-09-28
6019,1737002,0,63,2.0,22.48,1,NaN,1,2009-03-04,2011-10-12
6020,1737002,0,63,2.0,23.24,1,NaN,1,2009-03-04,2011-10-26
6021,1737002,0,63,2.0,23.05,1,NaN,1,2009-03-04,2011-11-09


In [9]:
# 위암의 경우, 위암 최초 진단 직전 건강검진 기록만 남기기로 함. 
# 위암 진단 전의 건강 검진이 없는 경우, 추후 삭제

new_c = pd.DataFrame(columns=['person_id', 'gender', 'age', 'EGD', 'bmi', 'alcohol_habit','smoking status', 'cancer', 'date', 'observation_date'])
persons=list(set(pre_df['person_id'].tolist()))

for p in persons:
    days=[]
    person_data = pre_df[pre_df['person_id']==p]
    
    for i in range(len(person_data)):
        cancer_date = pd.to_datetime(person_data.iloc[i,8]) # date
        check_date = pd.to_datetime(person_data.iloc[i,9]) #observation_date
        day = (cancer_date-check_date).days
        if day>0:
            days.append(99999)
        else:
            days.append(abs(day))
    min_date = days.index(min(days))
    #person_data.iloc[min_date,15]=cancer_date.year
    new_c = new_c.append(person_data.iloc[min_date,:])

In [10]:
print(len(persons))
print(len(new_c))

121
121


In [11]:
# 위암 이전 건강 검진 없는 경우, 삭제

new_c['date'] = pd.to_datetime(new_c['date'])
new_c['observation_date'] = pd.to_datetime(new_c['observation_date'])
new_c['diff'] = (new_c['date']-new_c['observation_date'])

In [12]:
new_c['diff'] = new_c['diff'].astype('str')

In [13]:
new_c

,person_id,gender,age,EGD,bmi,alcohol_habit,smoking status,cancer,date,observation_date,diff
5983,1673221,0,52,1.0,25.72,1,3.0,1,2008-09-30,2008-09-30,0 days
5649,1289235,0,62,NaN,NaN,1,NaN,1,2014-08-29,2009-05-18,1929 days
3358,805909,0,62,2.0,22.20,4,NaN,1,2015-08-20,2016-09-27,-404 days
5999,1690134,0,73,2.0,NaN,2,3.0,1,2014-05-09,2006-11-18,2729 days
1452,364065,0,69,2.0,NaN,2,3.0,1,2008-02-22,2008-02-23,-1 days
...,...,...,...,...,...,...,...,...,...,...,...
5955,1613795,0,72,3.0,19.94,4,NaN,1,2014-10-28,2014-10-28,0 days
3425,893936,0,62,3.0,19.76,1,3.0,1,2017-06-07,2017-06-07,0 days
5981,1668087,0,85,2.0,18.99,4,NaN,1,2008-05-09,2008-05-09,0 days
2417,654331,0,50,NaN,22.25,4,NaN,1,2014-02-17,2012-10-02,503 days


In [16]:
index = []
for i in range(len(new_c)):
    if new_c.iloc[i,10][0]!='-':
        index.append(i)

In [17]:
#index

In [18]:
new_c1 = new_c.iloc[index,:]

In [19]:
new_c1

,person_id,gender,age,EGD,bmi,alcohol_habit,smoking status,cancer,date,observation_date,diff
5983,1673221,0,52,1.0,25.72,1,3.0,1,2008-09-30,2008-09-30,0 days
5649,1289235,0,62,NaN,NaN,1,NaN,1,2014-08-29,2009-05-18,1929 days
5999,1690134,0,73,2.0,NaN,2,3.0,1,2014-05-09,2006-11-18,2729 days
1526,530465,0,64,4.0,17.40,2,NaN,1,2015-03-10,2015-03-10,0 days
3008,737827,0,55,6.0,23.36,1,NaN,1,2014-04-22,2014-04-22,0 days
...,...,...,...,...,...,...,...,...,...,...,...
5955,1613795,0,72,3.0,19.94,4,NaN,1,2014-10-28,2014-10-28,0 days
3425,893936,0,62,3.0,19.76,1,3.0,1,2017-06-07,2017-06-07,0 days
5981,1668087,0,85,2.0,18.99,4,NaN,1,2008-05-09,2008-05-09,0 days
2417,654331,0,50,NaN,22.25,4,NaN,1,2014-02-17,2012-10-02,503 days


In [20]:
#new_c1.to_csv('notunder_kd_cancer.csv',mode='w',index=False)